In [205]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [206]:
def split_data(data): # Split data 90% training 10% testing. Assumes the first column is Y.
    data = np.array(data)
    m, n = data.shape
    np.random.shuffle(data)
    
    split_index = int(m * 0.9)
    
    data_train = data[:split_index].T
    Y_train = data_train[0]
    X_train = data_train[1:n] / (n - 1)

    data_test = data[split_index:].T
    Y_test = data_test[0]
    X_test = data_test[1:n] / (n - 1)
    
    return X_train, X_test, Y_train, Y_test

In [207]:
class NeuralNetwork:
    
    def __init__(self): # Initializes weights and biases
        np.random.seed(10)
        self.W1 = np.random.rand(2, 30) - 0.5
        self.b1 = np.random.rand(2, 1) - 0.5
        
        self.W2 = np.random.rand(2, 2) - 0.5
        self.b2 = np.random.rand(2, 1) - 0.5
        
    def __ReLU(self, Z):
        return np.maximum(Z, 0)
    
    def __softmax(self, Z):
        Z = Z.astype(float)
        A = np.exp(Z - np.max(Z)) / sum(np.exp(Z - np.max(Z)))
        return A
    
    def __forward(self, X):
        Z1 = self.W1.dot(X) + self.b1
        A1 = self.__ReLU(Z1)
        
        Z2 = self.W2.dot(A1) + self.b2
        A2 = self.__softmax(Z2)
        return Z1, A1, Z2, A2

    def __label_encoder(self, Y): # Encodes each unique value with a number
        encoder = dict(zip(np.unique(Y), range(len(np.unique(Y)))))
        labels = []
        for i in Y:
            labels.append(encoder[i])
        
        return np.array(labels)
    
    def __one_hot(self, Y):
        Y = self.__label_encoder(Y)
        one_hot_Y = np.zeros((Y.size, Y.max() + 1))
        one_hot_Y[np.arange(Y.size), Y] = 1
        one_hot_Y = one_hot_Y.T
        return one_hot_Y
    
    def __prime_ReLU_(self, Z):
        return Z > 0

    def __backward(self, Z1, A1, Z2, A2, X, Y, learning_rate):
        one_hot_Y = self.__one_hot(Y)
        m, n = data.shape

        dZ2 = A2 - one_hot_Y
        dW2 = 1 / m * dZ2.dot(A1.T)
        db2 = 1 / m * np.sum(dZ2)

        dZ1 = self.W2.T.dot(dZ2) * self.__prime_ReLU_(Z1)
        dW1 = 1 / m * dZ1.dot(X.T)
        db1 = 1 / m * np.sum(dZ1)
        
        self.W1 = self.W1 - learning_rate * dW1
        self.b1 = self.b2 - learning_rate * db1
        self.W2 = self.W2 - learning_rate * dW2  
        self.b2 = self.b2 - learning_rate * db2
                
    def __get_predictions(self, A2): # Find largest largest predicted probability
        return np.argmax(A2, 0)
    
    def __predict(self, X):
        _, _, _, A2 = self.__forward(X)
        return self.__get_predictions(A2)
    
    def __get_accuracy(self, predictions, Y):
        return np.sum(predictions == Y) / Y.size

    def fit(self, X, Y, learning_rate, epochs):
        accuracy = []
        
        for i in range(epochs):
            Z1, A1, Z2, A2 = self.__forward(X)
            self.__backward(Z1, A1, Z2, A2, X, Y, learning_rate)
            if i % 10 == 0:
                accuracy.append(self.__get_accuracy(self.__predict(X), self.__one_hot(Y)[0]))
        
        return accuracy
        
    def test(self, X, Y):
        return self.__get_accuracy(self.__predict(X), self.__one_hot(Y)[0])

In [208]:
if __name__ == '__main__' and '__file__' not in globals():
    cols = list(pd.read_csv("data.csv", nrows =1))
    data = pd.read_csv('data.csv', usecols=[i for i in cols if i != 'id' and i != 'Unnamed: 32'])
    
    X_train, X_test, Y_train, Y_test = split_data(data)
    
    nn = NeuralNetwork()
    print('Training Accuracy: ', nn.fit(X_train, Y_train, 0.001, 50))
    
    print('Testing Accuracy: ', nn.test(X_test, Y_test))

Training Accuracy:  [0.376953125, 0.376953125, 0.388671875, 0.44140625, 0.58984375]
Testing Accuracy:  0.7894736842105263
